In [1]:
# The ResNet50 model is imported from Keras website
# Modified from https://keras.io/applications/
# Usage examples for image classification models
# Classify ImageNet classes with ResNet50

from keras.applications.resnet50 import ResNet50 # load the model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import time

In [2]:
# Load the weights of the ResNet50 model from website
model = ResNet50(weights='imagenet')
# The weights file is saved here
# model = ResNet50(weights='C:\Users\yhkao\.keras\models\resnet50_weights_tf_dim_ordering_tf_kernels.h5')

In [3]:
# This part is for displaying the bar chart in a window
# modified from https://matplotlib.org/gallery/user_interfaces/embedding_in_tk_sgskip.html

import tkinter as tk
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

# Implement the default Matplotlib key bindings.
from matplotlib.backend_bases import key_press_handler
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import cv2
import PIL
from PIL import Image,ImageTk

In [4]:
def MakePredictionOut(img_1):
    #img_2 = img_1[0:480,0:480]
    img_3 = cv2.resize(img_1,(224,224),2)
    x_1 = image.img_to_array(img_3)
    x_2 = np.expand_dims(x_1, axis=0)
    x_3 = preprocess_input(x_2)
    preds = model.predict(x_3)
    results = decode_predictions(preds, top=3)
    Category = ()
    Probability = [];
    for i in range(np.shape(results[0])[0]):
        Category = Category+(results[0][i][1],)
        Probability = Probability+[results[0][i][2],]
    return Category, Probability

In [5]:
# 預測結果作圖
def plotResults(Category, Probability):
    fig, ax = plt.subplots(figsize=(7,720/72))
    ax.set_xlabel('Probability')
    ax.set_title('What is it?')
    y_pos = np.arange(len(Category))
    x_pos = np.linspace(0,1,3)
    ax.barh(y_pos,Probability,align='center',color='red',ecolor='black')
    ax.set_yticks(y_pos)
    ax.set_yticklabels([])
    ax.set_xticks(x_pos)
    for i, v in enumerate(Probability):
        ax.text(0.05, i+0.01, str(Category[i]), color='blue', fontweight='bold',fontsize=18)
        ax.invert_yaxis() # labels read top-to-bottom
    return fig

In [6]:
# figure 轉換成 array, http://www.icare.univ-lille1.fr/tutorials/convert_a_matplotlib_figure

def fig2data(fig):
    #Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    #return a numpy 3D array of RGBA values
    # draw the renderer
    fig.canvas.draw( )
    # Get the RGBA buffer from the figure
    w,h = fig.canvas.get_width_height()
    buf = np.frombuffer(fig.canvas.tostring_argb(),dtype=np.uint8)
    buf.shape = (h,w,4)
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll(buf,3,axis=2)
    return buf

In [7]:
# 主程式, 從 Video 讀取一張影像, 用 MakePredictionOut 做預測.
# 用 tk.Label.after 更新視窗內容, https://stackoverflow.com/questions/16366857/show-webcam-sequence-tkinter

cap = cv2.VideoCapture(0)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("Image Size: %d x %d" % (width, height))

root = tk.Tk()
root.bind('<Escape>', lambda e: root.quit())
lmain = tk.Label(root)
lmain.pack()

Image Size: 640 x 480


In [8]:
def show_frame():
    global tStart, tEnd
    tStart = time.time() #計時開始
    # 攝影機影像擷取
    ret, frame_1 = cap.read()
    frame_2 = frame_1[0:480,80:560]
    frame = cv2.resize(frame_2,(720,720),1)
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
   
    # 影像結果預測
    Category, Probability = MakePredictionOut(frame)
    
    # 結果作圖
    fig = plotResults(Category, Probability)
    buf = fig2data(fig)
    plt.close(fig)
    
    # 結合影像與結果
    finimage = np.append(cv2image,buf,axis=1)
    
    # 轉換為輸出影像
    img = PIL.Image.fromarray(finimage)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, show_frame) # update every 10 ms
    tEnd = time.time() #計時結束
    #print('It takes %f sec for each frame' % (tEnd - tStart))

show_frame()
root.mainloop()

# 在視窗右上角, 按 X 即可結束程式.
cap.release()
cv2.destroyAllWindows()
print('It takes %f sec for each frame' % (tEnd - tStart))

It takes 0.263028 sec for each frame


In [ ]:
# https://www.youtube.com/watch?v=cRPgWHQjVLY